In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.stats.diagnostic import acorr_ljungbox

In [ ]:
#Exercice A
#Question 1
mu, sigma, samples = 0, 0.5, 1000
x = np.random.normal(mu, sigma, samples)
plt.title('Bruit blanc gaussien de 1000 échantillons')
plt.xlabel('Temps')
plt.ylabel('Amplitude')
plt.plot(x)

In [ ]:
#Question 3
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(x, lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(x, lags=20, ax=ax2)

In [ ]:
#Question 4
adf = adfuller(x)
print(adf)

In [ ]:
test_kpss = kpss(x)
print(test_kpss)

In [ ]:
#Exercice B.1
#Question 1
sales = pd.read_csv('sales.csv')
print(sales['Date'][:48])

In [ ]:
plt.plot(sales['Date'][:48], sales['Fuel'][:48])
plt.plot(sales['Date'][:48], sales['Food'][:48])
plt.legend(['Carburant', 'Produits alimentaires'])
plt.xlabel('Date')
plt.ylabel('Chiffre d\'affaires')

In [ ]:
shapiro_fuel = stats.shapiro(sales['Fuel'][:48])
print(shapiro_fuel)
print(acorr_ljungbox(sales['Fuel'][:48], return_df=True))

In [ ]:
shapiro_food = stats.shapiro(sales['Food'][:48])
print(shapiro_food)
print(acorr_ljungbox(sales['Food'][:48], return_df=True))

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(sales['Fuel'][:48], lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(sales['Fuel'][:48], lags=20, ax=ax2)

In [ ]:
print(adfuller(sales['Fuel'][:48]))
print(kpss(sales['Fuel'][:48]))

In [ ]:
#Exercice B.2
#Question 1
diff_fuel = np.array([])
for i in range(1, 49, 1):
    diff = sales['Fuel'][i] - sales['Fuel'][i-1]
    diff_fuel = np.append(diff_fuel, diff)

In [ ]:
plt.plot(sales['Date'][:47], diff_fuel)
plt.title('Différenciation de la série du carburant')
plt.xlabel('Date')
plt.ylabel('Différence')

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(diff_fuel, lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(diff_fuel, lags=20, ax=ax2)

In [ ]:
print(adfuller(diff_fuel))
print(kpss(diff_fuel))

In [ ]:
#Exercice C
#Question 1
x_train_food = np.array([])
for i in range(1, 49, 1):
    diff = sales['Food'][i] - sales['Food'][i-1]
    x_train_food = np.append(x_train_food, diff)

In [ ]:
x_train_fuel = np.array([])
for i in range(1, 49, 1):
    diff = sales['Fuel'][i] - sales['Fuel'][i-1]
    diff_fuel = np.append(x_train_fuel, diff)

In [ ]:
x_test_food = np.array([])
for i in range(48, 50, 1):
    diff = sales['Food'][i] - sales['Food'][i-1]
    x_test_diff = np.append(x_test_diff, diff)
print(len(x_test_food))

In [ ]:
x_test_fuel = np.array([])
for i in range(48, 50, 1):
    diff = sales['Fuel'][i] - sales['Fuel'][i-1]
    x_test_diff = np.append(x_test_diff, diff)

In [ ]:
#Question 2
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(x_train_food, lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(x_train_food, lags=20, ax=ax2)

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(x_train_diff, lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(x_train_diff, lags=20, ax=ax2)

In [ ]:
#Question 3
model = ARIMA(x_train_food, order=(1,1,1)).fit()
print(ARIMA(x_test_food, order=(1,1,1)).fit().summary())

In [ ]:
#Question 4
resid = ARIMA(x_train_food, order=(1,1,1)).fit().resid
print(resid)
print(stats.shapiro(resid))
print(kpss(resid))
print(adfuller(resid))
print((acorr_ljungbox(resid, return_df=True)))

In [ ]:
#Question 7
x_train_food = np.append(x_train_food, ARIMA(x_train_food, order=(1,1,1)).fit().forecast())

In [ ]:
model = ARIMA(x_train_food, order=(1,1,1)).fit()

In [ ]:
x_train_food = np.append(x_train_food, ARIMA(x_train_food, order=(1,1,1)).fit().forecast())

In [ ]:
plt.plot(sales['Dates'][:48], x_train_food[:48], 'crimson', label="Train")
plt.plot(sales['Dates'][47:50], x_train_food[47:50], 'navy', label="Prédictions")
plt.plot(sales['Dates'][47:50], x_test_food, 'steelblue', label="Test")
plt.title("Comparaison prédictions-valeurs attendues")
plt.legend()
plt.show()

In [ ]:
mse = np.sum( (x_train_food[48:50]- [1.74, 12.21])**2 ) / 2
print('mse: ', np.sqrt(mse))